In [1]:
import logging
import os
import math
import shutil
import subprocess
from pathlib import Path
from collections import Counter

import tifffile
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF
import torch
from skimage.filters import gaussian
from skimage.morphology import binary_erosion, remove_small_objects, binary_dilation, label
from skimage.measure import regionprops_table, regionprops
from scipy.ndimage import binary_fill_holes
from skimage.segmentation import expand_labels
from skimage.exposure import rescale_intensity
from einops import rearrange


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import multiplex_imaging_pipeline.utils as utils
from multiplex_imaging_pipeline.region_features import get_region_features

In [5]:
output_dir = '/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1'
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [6]:
adata_fps = sorted(utils.listfiles(os.path.join(output_dir, 'spatial_features'), regex=r'.h5ad$'))
sample_to_adata = {fp.split('/')[-1].replace('.h5ad', ''):fp for fp in adata_fps}

In [7]:
ome_fps = sorted(utils.listfiles('/diskmnt/Projects/Users/estorrs/multiplex_data/codex/htan',
                         regex='/level_2/[^/]+.ome.tiff$'))
sample_to_ome = {fp.split('/')[-1].replace('.ome.tiff', ''):fp for fp in ome_fps}

In [8]:
sample_to_adata.keys()

dict_keys(['HT110B1_S1H4', 'HT171B1-S1H9A1-4_left_05122023', 'HT171B1-S1H9A1-4_right_05122023', 'HT171B1-S1H9A1-A4_left_05262023', 'HT171B1-S1H9A1-A4_right_05262023', 'HT206B1-S1H5_U1_07012023', 'HT206B1-S1H5_U2_07012023', 'HT206B1_H1', 'HT206B1_H1_06252022', 'HT206B1_S1H2L4_20221028', 'HT243B1-S1H4A4_04192023', 'HT243B1-S1H4A4_left_05122023', 'HT243B1-S1H4A4_left_05262023', 'HT243B1-S1H4A4_right_05122023', 'HT243B1-S1H4A4_right_05262023', 'HT243B1_S1H4', 'HT271B1-S1H6A5_04192023', 'HT271B1-S1H6A5_left_05122023', 'HT271B1-S1H6A5_left_05262023', 'HT271B1-S1H6A5_mid_05262023', 'HT271B1-S1H6A5_right_05122023', 'HT271B1-S1H6A5_right_05262023', 'HT271B1_S1H3A5_02172023', 'HT297B1-S1H1_U1_07012023', 'HT297B1-S1H1_U2_07012023', 'HT305B1_S1H1', 'HT308B1-S1H5A4_04192023', 'HT308B1-S1H5A4_left_05122023', 'HT308B1-S1H5A4_left_05262023', 'HT308B1-S1H5A4_right_05122023', 'HT308B1-S1H5A4_right_05262023', 'HT323B1_H1A1', 'HT323B1_H1A4', 'HT323B1_H1_08042022', 'HT323B1_H3', 'HT339B1_H1A1', 'HT339B1_H2

In [9]:
sorted(sample_to_ome.keys())

['20230120_PKD_K1301459',
 '20230120_PKD_K200452_2PB',
 '20230120_PKD_K2200446_4PB',
 'CRC112',
 'E16.5_female_section1_04272023',
 'E16.5_female_section2_04272023',
 'E16.5_male_section1_04272023',
 'E16.5_male_section2_04272023',
 'E165_F4LM2L_left_07192023',
 'E165_F4LM2L_right_07192023',
 'E165_F6RM3R_left_07202023',
 'E165_F6RM3R_right_07202023',
 'HS-18-11250-1B_06102023',
 'HS-18-6934-2A_06062023',
 'HS-19-14816-1J_06062023',
 'HS-19-6647-1B_06102023',
 'HT110B1-S1H4',
 'HT171B1-S1H1A1',
 'HT171B1-S1H1A4',
 'HT171B1-S1H9A1-4_04192023',
 'HT171B1-S1H9A1-4_left_05122023',
 'HT171B1-S1H9A1-4_right_05122023',
 'HT171B1-S1H9A1-A4_left_05262023',
 'HT171B1-S1H9A1-A4_right_05262023',
 'HT171B1_20221205',
 'HT171B1_S1H1A1',
 'HT171B1_S1H1A4',
 'HT171B1_S1H8A1_20221122',
 'HT206B1-H1',
 'HT206B1-S1H5_U1_07012023',
 'HT206B1-S1H5_U2_07012023',
 'HT206B1_Ctrl',
 'HT206B1_H1_06252022',
 'HT206B1_S1H2L4',
 'HT243B1-S1H4',
 'HT243B1-S1H4A4_04192023',
 'HT243B1-S1H4A4_left_05122023',
 'HT243B1

In [10]:
set(sample_to_adata.keys()).difference(set(sample_to_ome.keys()))

{'HT110B1_S1H4',
 'HT206B1_H1',
 'HT206B1_S1H2L4_20221028',
 'HT243B1_S1H4',
 'HT271B1_S1H3A5_02172023',
 'HT305B1_S1H1',
 'HT323B1_H1A1',
 'HT323B1_H1A4',
 'HT323B1_H1_08042022',
 'HT323B1_H3',
 'HT339B1_H1A1',
 'HT339B1_H2A1',
 'HT365B1_S1H1_02132023',
 'HT397B1_H2A2',
 'HT397B1_H3A1',
 'HT480B1_S1H2_R001',
 'HT480B1_S1H2_R002'}

In [11]:
name_map = {
    'HT110B1-S1H4': 'HT110B1_S1H4',
    'HT206B1-H1': 'HT206B1_H1',
    'HT206B1_S1H2L4': 'HT206B1_S1H2L4_20221028',
    'HT243B1-S1H4': 'HT243B1_S1H4',
    'HT271B1-S1H3A5': 'HT271B1_S1H3A5_02172023',
    'HT305B1-S1H1': 'HT305B1_S1H1',
    'HT323B1-H1A1': 'HT323B1_H1A1',
    'HT323B1-H1A4': 'HT323B1_H1A4',
    'HT323B1-H1-08042022': 'HT323B1_H1_08042022',
    'HT323B1-H3': 'HT323B1_H3',
    'HT339B1-H1A1': 'HT339B1_H1A1',
    'HT339B1-H2A1': 'HT339B1_H2A1',
    'HT365B1_S1H1': 'HT365B1_S1H1_02132023',
    'HT397B1-H2A2': 'HT397B1_H2A2',
    'HT397B1-H3A1': 'HT397B1_H3A1',
    'HT480B1-S1H2-R001': 'HT480B1_S1H2_R001',
    'HT480B1-S1H2-R002': 'HT480B1_S1H2_R002'
}

sample_to_ome = {name_map.get(k, k):v for k, v in sample_to_ome.items()}

#### only run to generate masks

In [17]:
def generate_mask(mask, sigma=1., min_area=10000):
    mask = gaussian(mask, sigma=sigma)
    mask = binary_fill_holes(mask)
    
    labeled = label(mask)
    props = regionprops(labeled)
    
    mask = labeled > 0
    for prop in props:
        if prop.area < min_area:
            r1, c1, r2, c2 = prop.bbox
            r1, c1 = max(r1 - 10, 0), max(c1 - 10, 0)
            r2, c2, = r2 + 10, c2 + 10
            mask[r1:r2, c1:c2][labeled[r1:r2, c1:c2]==prop.label] = 0
            
    return mask

In [18]:
out = os.path.join(output_dir, 'region_masks')
Path(out).mkdir(parents=True, exist_ok=True)

In [21]:
keep = ['Pan-Cytokeratin', 'E-cadherin']
sample_to_mask = {}
for sample in sample_to_adata.keys():
    print(sample)
    channel_to_img = utils.extract_ome_tiff(sample_to_ome[sample])
    channel_to_img = {utils.R_CHANNEL_MAPPING.get(k, k):img for k, img in channel_to_img.items()}

    a = sc.read_h5ad(sample_to_adata[sample])
    
    markers = [x.replace('_fraction', '') for x in a.var.index.to_list()]
    thresholds = a.uns['thresholds']
    mask = np.zeros_like(next(iter(channel_to_img.values())), dtype=bool)
    for c in keep:
        if c in markers:
            val = thresholds[markers.index(c)]
            if val > 0:
                m = channel_to_img[c] >= val
                mask |= m
        
    mask = generate_mask(mask, sigma=1., min_area=10000)
    sample_to_mask[sample] = mask
    
    tifffile.imsave(os.path.join(out, f'{sample}.tif'), mask)    

HT110B1_S1H4


/diskmnt/Projects/Users/estorrs/miniconda3/envs/mip/lib/python3.8/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
/diskmnt/Projects/Users/estorrs/sandbox/tmp/ipykernel_214946/578118905.py:23: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(os.path.join(out, f'{sample}.tif'), mask)


HT171B1-S1H9A1-4_left_05122023


/diskmnt/Projects/Users/estorrs/miniconda3/envs/mip/lib/python3.8/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
/diskmnt/Projects/Users/estorrs/sandbox/tmp/ipykernel_214946/578118905.py:23: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(os.path.join(out, f'{sample}.tif'), mask)
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mip/lib/python3.8/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (t

HT171B1-S1H9A1-4_right_05122023


...

/diskmnt/Projects/Users/estorrs/sandbox/tmp/ipykernel_214946/578118905.py:23: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(os.path.join(out, f'{sample}.tif'), mask)
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mip/lib/python3.8/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


HT565B1-S1H2_U1_07012023


/diskmnt/Projects/Users/estorrs/sandbox/tmp/ipykernel_214946/578118905.py:23: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(os.path.join(out, f'{sample}.tif'), mask)
/diskmnt/Projects/Users/estorrs/miniconda3/envs/mip/lib/python3.8/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


HT565B1-S1H2_U2_07012023


/diskmnt/Projects/Users/estorrs/sandbox/tmp/ipykernel_214946/578118905.py:23: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(os.path.join(out, f'{sample}.tif'), mask)


#### start here to rerun pregen masks

In [12]:
mask_fps = sorted(utils.listfiles(os.path.join(output_dir, 'region_masks'), regex=r'.tif$'))
sample_to_mask = {fp.split('/')[-1].replace('.tif', ''):fp for fp in mask_fps}

In [13]:
a = set(sample_to_adata.keys())
b = set(sample_to_mask.keys())
c = set(sample_to_ome.keys())

len(a.intersection(b).intersection(c))

51

In [14]:
a = sc.read_h5ad(next(iter(sample_to_adata.values())))
a

AnnData object with n_obs × n_vars = 77651 × 26
    obs: 'row', 'col', 'bbox-r1', 'bbox-c1', 'bbox-r2', 'bbox-c2', 'area', 'DAPI_intensity', 'CD8_intensity', 'CD138_intensity', 'CD4_intensity', 'cKit_intensity', 'Pan-Cytokeratin_intensity', 'GLUT1_intensity', 'Podoplanin_intensity', 'CK17_intensity', 'HLA-DR_intensity', 'GATA3_intensity', 'CD36_intensity', 'MGP_intensity', 'CK14_intensity', 'SMA_intensity', 'CD68_intensity', 'CD20_intensity', 'Vimentin_intensity', 'CD3e_intensity', 'CD163_intensity', 'Ki67_intensity', 'PR_intensity', 'ER_intensity', 'P16_intensity', 'CK19_intensity', 'P21_intensity', 'default_annotation', 'cKit_annotation', 'GATA3_annotation', 'CK14_annotation', 'CK19_annotation', 'PR_annotation', 'ER_annotation', 'Her2_annotation', 'CD8_annotation', 'BCA1_annotation', 'CTLA4_annotation', 'CD68_annotation', 'CD163_annotation', 'Granzyme-B_annotation', 'pan-immune_annotation'
    uns: 'gating_strategy', 'thresholds'
    obsm: 'spatial'

In [15]:
annotation_keys = [c for c in a.obs.columns if '_annotation' in c]
annotation_keys

['default_annotation',
 'cKit_annotation',
 'GATA3_annotation',
 'CK14_annotation',
 'CK19_annotation',
 'PR_annotation',
 'ER_annotation',
 'Her2_annotation',
 'CD8_annotation',
 'BCA1_annotation',
 'CTLA4_annotation',
 'CD68_annotation',
 'CD163_annotation',
 'Granzyme-B_annotation',
 'pan-immune_annotation']

In [16]:
region_features_dir = os.path.join(output_dir, 'region_features')
region_features_masks_dir = os.path.join(output_dir, 'region_features_masks')
Path(region_features_dir).mkdir(parents=True, exist_ok=True)
Path(region_features_masks_dir).mkdir(parents=True, exist_ok=True)

In [17]:
logging.getLogger().setLevel(logging.INFO)

In [18]:
for sample in sample_to_adata.keys():
    print(sample)
    try:
        combined, labeled_dict = get_region_features(
            sample_to_adata[sample], sample_to_ome[sample], mask_fp=sample_to_mask[sample],
            annotation_keys=annotation_keys
        )
        combined.index.name = 'label'
        combined.to_csv(os.path.join(region_features_dir, f'{sample}.txt'), sep='\t', index=True)

        for name, img in labeled_dict.items():
            tifffile.imwrite(os.path.join(region_features_masks_dir, f'{sample}_{name}.tif'), img, compression='LZW')
    except:
        print(f'error in sample {sample}')

HT110B1_S1H4


/diskmnt/Projects/Users/estorrs/miniconda3/envs/mip/lib/python3.8/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
INFO:root:reading mask from /diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_masks/HT110B1_S1H4.tif
INFO:root:generating region features for region
INFO:root:generating cell fractions
INFO:root:generating marker intensities
INFO:root:generating marker fractions
INFO:root:Using the following threhsolds to calculate positive fraction: {'CD8': 11.0, 'Pan-Cytokeratin': 10.0, 'Podoplanin': 10.0, 'CK17': 20.0, 'HLA-DR': 12.0, 'GATA3': 9.0, 'CK14': 19.0, 'SMA': 15.0, 'CD20': 12.0, 'CD3e': 15.0, 'CD163': 19.0, 'PR': 39.0, 

HT171B1-S1H9A1-4_left_05122023


/diskmnt/Projects/Users/estorrs/miniconda3/envs/mip/lib/python3.8/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
INFO:root:reading mask from /diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_masks/HT171B1-S1H9A1-4_left_05122023.tif
INFO:root:generating region features for region
INFO:root:generating cell fractions
INFO:root:generating marker intensities
INFO:root:generating marker fractions
INFO:root:Using the following threhsolds to calculate positive fraction: {'CD8': 10.0, 'Her2': 10.0, 'GATA3': 29.0, 'cKit': 10.0, 'Pan-Cytokeratin': 15.0, 'GLUT1': 6.0, 'Podoplanin': 10.0, 'HLA-DR': 12.0, 'CK14': 10.0, 'E-cadherin': 212.0, 

HT171B1-S1H9A1-4_right_05122023


...

/diskmnt/Projects/Users/estorrs/miniconda3/envs/mip/lib/python3.8/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
INFO:root:reading mask from /diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_masks/HT565B1-H2_04262023.tif
INFO:root:generating region features for region
INFO:root:generating cell fractions
INFO:root:generating marker intensities
INFO:root:generating marker fractions
INFO:root:Using the following threhsolds to calculate positive fraction: {'CD8': 10.0, 'Her2': 10.0, 'GATA3': 19.0, 'cKit': 10.0, 'Pan-Cytokeratin': 12.0, 'GLUT1': 15.0, 'Podoplanin': 10.0, 'HLA-DR': 12.0, 'SMA': 10.0, 'CD3e': 34.0, 'PR': 49.0, 'ER': 

HT565B1-S1H2_U1_07012023


/diskmnt/Projects/Users/estorrs/miniconda3/envs/mip/lib/python3.8/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
INFO:root:reading mask from /diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_masks/HT565B1-S1H2_U1_07012023.tif
INFO:root:generating region features for region
INFO:root:generating cell fractions
INFO:root:generating marker intensities
INFO:root:generating marker fractions
INFO:root:Using the following threhsolds to calculate positive fraction: {'CD8': 10.0, 'Her2': 15.0, 'GATA3': 40.0, 'cKit': 10.0, 'GLUT1': 10.0, 'Podoplanin': 10.0, 'HLA-DR': 12.0, 'CK14': 12.0, 'E-cadherin': 10.0, 'SMA': 10.0, 'CD3e': 39.0, 'CD3

HT565B1-S1H2_U2_07012023


/diskmnt/Projects/Users/estorrs/miniconda3/envs/mip/lib/python3.8/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
INFO:root:reading mask from /diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_masks/HT565B1-S1H2_U2_07012023.tif
INFO:root:generating region features for region
INFO:root:generating cell fractions
INFO:root:generating marker intensities
INFO:root:generating marker fractions
INFO:root:Using the following threhsolds to calculate positive fraction: {'CD8': 10.0, 'Her2': 15.0, 'GATA3': 40.0, 'cKit': 10.0, 'GLUT1': 10.0, 'Podoplanin': 10.0, 'HLA-DR': 12.0, 'CK14': 12.0, 'E-cadherin': 10.0, 'SMA': 10.0, 'CD3e': 39.0, 'CD3

In [ ]:
df = pd.read_csv(
    '/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features/HT110B1_S1H4.txt',
    sep='\t'
)
df

In [ ]:
list(df.columns)

In [ ]:
out = os.path.join(output_dir, 'pseudo_fullres')
Path(out).mkdir(parents=True, exist_ok=True)

In [96]:
cmap = {
    'Pan-Cytokeratin': (1., 0., 0.),
    'SMA': (0., 1., 0.),
    'DAPI': (0., 0., 1.),
    'Podoplanin': (1., 1., 1.),    
}

for sample in sample_to_adata.keys():
    print(sample)
    fp = sample_to_ome[sample]
    channel_to_img = utils.extract_ome_tiff(fp, channels=list(cmap.keys()), flexibility='loose')

    pseudo = utils.make_pseudo(channel_to_img, cmap=cmap, contrast_pct=95.)
    pseudo *= 255.
    pseudo = pseudo.astype(np.uint8)
    
    tifffile.imwrite(os.path.join(out, f'{sample}.tif'), pseudo, compression='LZW')    

HT110B1_S1H4


/diskmnt/Projects/Users/estorrs/miniconda3/envs/mip/lib/python3.8/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


HT171B1-S1H9A1-4_left_05122023
HT171B1-S1H9A1-4_right_05122023
HT171B1-S1H9A1-A4_left_05262023
HT171B1-S1H9A1-A4_right_05262023
HT206B1-S1H5_U1_07012023
HT206B1-S1H5_U2_07012023
HT206B1_H1
HT206B1_H1_06252022
HT206B1_S1H2L4_20221028
HT243B1-S1H4A4_04192023
HT243B1-S1H4A4_left_05122023
HT243B1-S1H4A4_left_05262023
HT243B1-S1H4A4_right_05122023
HT243B1-S1H4A4_right_05262023
HT243B1_S1H4
HT271B1-S1H6A5_04192023
HT271B1-S1H6A5_left_05122023
HT271B1-S1H6A5_left_05262023
HT271B1-S1H6A5_mid_05262023
HT271B1-S1H6A5_right_05122023
HT271B1-S1H6A5_right_05262023
HT271B1_S1H3A5_02172023
HT297B1-S1H1_U1_07012023
HT297B1-S1H1_U2_07012023
HT305B1_S1H1
HT308B1-S1H5A4_04192023
HT308B1-S1H5A4_left_05122023
HT308B1-S1H5A4_left_05262023
HT308B1-S1H5A4_right_05122023
HT308B1-S1H5A4_right_05262023
HT323B1_H1A1
HT323B1_H1A4
HT323B1_H1_08042022
HT323B1_H3
HT339B1_H1A1
HT339B1_H2A1
HT339B2-H2_U1_07012023
HT339B2-H2_U2_07012023
HT365B1_S1H1_02132023
HT397B1_H2A2
HT397B1_H3A1
HT397B1_S1H1A3U22_04122023
HT397B1_S

making thumbnails

In [69]:
scale = .05

In [70]:
mask_fps = sorted(utils.listfiles(os.path.join(output_dir, 'region_features_masks'), regex=r'region.*.tif$'))
mask_fps = [fp for fp in mask_fps if 'thumbnail' not in fp]

In [74]:
for fp in mask_fps:
    print(fp)
    new_fp = fp.replace('.tif', '_thumbnail_05.tif')
    img = tifffile.imread(fp)
    img = torch.tensor(img).unsqueeze(0)
    thumb = TF.resize(img, (int(img.shape[-2] * scale), int(img.shape[-1] * scale)),
                     interpolation=TF.InterpolationMode.NEAREST, antialias=False)
    thumb = thumb.squeeze().numpy()
    tifffile.imwrite(new_fp, thumb, compression='LZW')

/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT110B1_S1H4_boundary.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT110B1_S1H4_ductal.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT110B1_S1H4_expanded.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT110B1_S1H4_myoepi.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT110B1_S1H4_region.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT171B1-S1H9A1-4_left_05122023_boundary.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT171B1-S1H9A1-4_left_05122023_ductal.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_

/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT243B1-S1H4A4_left_05122023_region.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT243B1-S1H4A4_left_05262023_boundary.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT243B1-S1H4A4_left_05262023_ductal.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT243B1-S1H4A4_left_05262023_expanded.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT243B1-S1H4A4_left_05262023_myoepi.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT243B1-S1H4A4_left_05262023_region.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT243B1-S1H4A4_right_0512202

/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT297B1-S1H1_U2_07012023_expanded.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT297B1-S1H1_U2_07012023_myoepi.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT297B1-S1H1_U2_07012023_region.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT305B1_S1H1_boundary.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT305B1_S1H1_ductal.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT305B1_S1H1_expanded.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT305B1_S1H1_myoepi.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_

/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT339B1_H2A1_myoepi.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT339B1_H2A1_region.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT339B2-H2_U1_07012023_boundary.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT339B2-H2_U1_07012023_ductal.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT339B2-H2_U1_07012023_expanded.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT339B2-H2_U1_07012023_myoepi.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT339B2-H2_U1_07012023_region.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/r

/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT565B1-H2_04262023_region.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT565B1-S1H2_U1_07012023_boundary.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT565B1-S1H2_U1_07012023_ductal.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT565B1-S1H2_U1_07012023_expanded.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT565B1-S1H2_U1_07012023_myoepi.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT565B1-S1H2_U1_07012023_region.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1/region_features_masks/HT565B1-S1H2_U2_07012023_boundary.tif
/diskmnt/Projects/U